In [ ]:
from my_packages import *

#### check dates with satisfying number of bursts (= 9)

In [ ]:
fpath = r"D:\SharedFolder\CRFP_desc_burst_2014_2024_v2.xlsx"
df = pd.read_excel(fpath)
df = df.sort_values(by="startTime").reset_index(drop=True)

In [ ]:
df["Acquisitions"] = [pd.to_datetime(ele.split("T")[0]) for ele in df["startTime"]]

In [ ]:
unique_acquisition = sorted(set(df["Acquisitions"].tolist()))
unique_acquisition[:3], len(unique_acquisition)

#### check the unique num. of bursts for each acquisition

In [ ]:
cache = {}
for TIME in unique_acquisition:
    scene_num = len(df.query("Acquisitions==@TIME"))
    scene_num_str = str(scene_num)
    if scene_num_str not in cache.keys():
        cache[scene_num_str] = []

    cache[scene_num_str].append(TIME)

count_dict = {}
for key, values in cache.items():
    count_dict[key] = len(values)

print(count_dict)

#### images pairs - sequential = 4

In [ ]:
temp = pd.DataFrame(data=cache["9"] + cache["10"], columns=["Acquisitions"])
temp = temp.sort_values(by="Acquisitions").reset_index(drop=True)
temp.head(3)

In [ ]:
cache = {
    "Reference": [],
    "Secondary": [],
    "ExpectedPairs": [],
    "Intervals": [],
}

num_images_to_pair = 4

for select_date in temp.iloc[:, 0]:
    ref = select_date.strftime("%Y%m%d")
    for secondary in temp.query("Acquisitions>@select_date")[:num_images_to_pair]["Acquisitions"]:
        sec = secondary.strftime("%Y%m%d")
        cache["Reference"].append(select_date)
        cache["Secondary"].append(secondary)
        cache["ExpectedPairs"].append(f"{ref}_{sec}")
        cache["Intervals"].append((secondary - select_date).days)

#### check current pairs

In [ ]:
text_fld = r"E:\001_InSAR_Project\9000_S1A_DESCENDING\temp2"
files = os.listdir(text_fld)

cache = {"Reference": [], "Secondary": [], "Pairs": [], "Intervals": []}

for ele in files:
    ref_str = ele.split("_")[1]
    sec_str = ele.split("_")[2]
    ref = pd.to_datetime(ref_str)
    sec = pd.to_datetime(sec_str)
    pair = f"{ref_str}_{sec_str}"
    date_intervals = (sec - ref).days
    for k, v in zip(cache.keys(), [ref, sec, pair, date_intervals]):
        cache[k].append(v)

desc_pair_df = pd.DataFrame(cache)
desc_pair_df = desc_pair_df.sort_values("Pairs").reset_index(drop=True)
desc_pair_df.head(3)

In [ ]:
count_dict = {}

for ref in desc_pair_df["Reference"].unique():

    scene_num = len(desc_pair_df.query("Reference==@ref"))
    scene_num_str = str(scene_num)

    if scene_num_str not in count_dict.keys():
        count_dict[scene_num_str] = []

    ref_datetime = ref.astype("datetime64[s]").astype(datetime)
    count_dict[scene_num_str].append(ref_datetime.strftime("%Y%m%d"))

count_pairnum = {}
for key, values in count_dict.items():
    count_pairnum[key] = len(values)

print(count_pairnum)

#### check completedly-processed output descending ifgrams

In [ ]:
mainfolder = r"K:\PROJECT\001_CHOUSHUI_RIVER_BASIN\200_CRFP_S1A_HYP3\CRFP_desc\HYP3_OUTPUT"

folders = sorted(glob(os.path.join(mainfolder, "S*")))
folders[:3]